# Branching Process Example

This notebook demonstrates a simple branching process using the Gillespie algorithm.

A branching process models population growth where each individual can:
- Give birth to offspring at rate λ
- Die at rate μ

This uses the same AbstractSystem and AbstractAlgorithm framework as equilibrium simulations.

In [1]:
using Random
using MonteCarloX

## Define the System

In [2]:
# Birth-Death process system
mutable struct BranchingProcess <: AbstractSystem
    N::Int              # Population size
    λ::Float64          # Birth rate per individual
    μ::Float64          # Death rate per individual
    rates::Vector{Float64}  # [birth_rate, death_rate]
    
    function BranchingProcess(N0::Int, λ::Float64, μ::Float64)
        rates = [λ * N0, μ * N0]
        new(N0, λ, μ, rates)
    end
end

In [3]:
# Define how the system changes
function modify!(sys::BranchingProcess, event::Int)
    if event == 1  # Birth
        sys.N += 1
    elseif event == 2  # Death
        sys.N -= 1
    end
    # Update rates
    sys.rates[1] = sys.λ * sys.N
    sys.rates[2] = sys.μ * sys.N
    return nothing
end

modify! (generic function with 1 method)

## Setup and Run Simulation

In [4]:
# Create system
rng = MersenneTwister(42)
sys = BranchingProcess(10, 0.42, 0.4)  # N0=10, λ=0.42, μ=0.4

# Create Gillespie algorithm
sim = init(rng, Gillespie(), sys.rates)

println("Initial population: $(sys.N)")
println("Birth rate λ: $(sys.λ)")
println("Death rate μ: $(sys.μ)")

Initial population: 10
Birth rate λ: 0.42
Death rate μ: 0.4


In [5]:
# Setup measurements
T = 100.0
measurements = Measurements([
    :population => (s -> s.N) => Float64[]
], collect(0:0.5:T))

# Initial measurement
t = 0.0
measure!(measurements, sys, t)

In [6]:
# Run simulation
while t <= T && sys.N > 0
    dt, event = next(sim)
    t += dt
    
    # Measure before modifying
    measure!(measurements, sys, t)
    
    # Update system
    modify!(sys, event)
    
    # Update rates in simulation
    sim.event_handler.list_rate[:] = sys.rates
end

println("\nSimulation complete")
println("Final time: $t")
println("Final population: $(sys.N)")


Simulation complete
Final time: 31.146289184274302
Final population: 0


## Analyze Results

In [8]:
using StatsBase

population_data = measurements[:population].data

println("\nPopulation statistics:")
println("  Mean: $(round(mean(population_data), digits=2))")
println("  Max: $(maximum(population_data))")
println("  Min: $(minimum(population_data))")

# Expected behavior: if λ > μ, population grows; if λ < μ, population declines
if sys.λ > sys.μ
    println("\nλ > μ: Expected population growth")
else
    println("\nλ < μ: Expected population decline")
end


Population statistics:
  Mean: 12.49
  Max: 31.0
  Min: 2.0

λ > μ: Expected population growth
